In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# kippenhahn

In [2]:
overshoot = 0.0
init_mass_arr = np.arange(70, 141, 10)
CO_mass_core_arr = np.full(init_mass_arr.shape, np.nan)
for i, init_mass in enumerate(init_mass_arr):
    datadir = Path('/home/jiangrz/hdd23/INAC24G3/Over_%02d_finer/%03d'%(overshoot, init_mass))
    model_info = pd.DataFrame(columns=['name', 'age'], dtype=float)
    model_names = []
    for _path in datadir.iterdir():
        if (_path.suffix == '.atm'):
            model_names.append(_path.stem)
    # fort54 = pd.read_csv(datadir/'fort.54', delimiter='\s+', header=None)
    model_names.sort()
    model_info['name'] = model_names
    y_gridnum = 2500
    y_grid = np.linspace(0, init_mass, y_gridnum)
    y_mesh = np.diff(y_grid)/2+y_grid[:-1]
    E_arr = np.full((len(model_info), len(y_mesh)), np.nan)
    RADDAD_arr = np.full((len(model_info), len(y_mesh)), np.nan)
    for row_idx, row in model_info.iterrows():
        model_name = row['name']
        df_f01 = pd.read_csv(
            datadir/('%s.f01'%model_name), delimiter='\s+', 
            usecols=[1, 6, 18]).reset_index(drop=True)
        df_f01.loc[len(df_f01)+1] = np.full(3, np.nan)
        df_chi = pd.read_csv(
            datadir/('%s.chi'%model_name), delimiter='\s+', 
            usecols=[1])
        df_qva = pd.read_csv(
            datadir/('%s.qva'%model_name), delimiter='\s+', 
        )
        age = df_qva.loc[0, 'AGE']
        model_info.loc[row_idx, 'age'] = age
        E_arr[row_idx, :len(df_f01)] = df_f01['M/MTOT'].values
        RADDAD_arr[row_idx, :len(df_f01)] = np.sign(df_f01['RAD-DAD'])
        # RADDAD_arr[RADDAD_arr < 1] = np.nan
        adopt_idx = np.sum((y_mesh >= df_chi['M'].values.reshape(-1, 1)), axis=0)-1
        adopt_E = df_f01['E-TOT'].values[adopt_idx]
        E_arr[row_idx, :] = adopt_E
    tau = model_info['age'].values[-1]-model_info['age'].values
    tau[0] /= 2
    tau[-1] /= 2
    model_info['logt'] = np.log10(model_info['age'].values[-1]-model_info['age'].values)
    x_mesh = model_info['logt'].values
    mgrid = np.meshgrid(x_mesh, y_mesh)
    
    # mass_mesh = datadir
    # df_keyproperty = pd.DataFrame(columns=['model', 'time', ], dtype=(str, float))
    x_meshdiff = np.diff(x_mesh)
    x_grid = np.concatenate(([x_mesh[0]-x_meshdiff[0]/2], x_meshdiff/2+x_mesh[:-1], [x_mesh[-1]+x_meshdiff[-1]/2]))
    signlogE_arr = np.sign(E_arr.T)*np.log10(np.abs(E_arr.T))

    plt.figure(figsize=(8, 6), dpi=150)
    X, Y = np.meshgrid(x_grid, y_grid)
    vmin = -10 #np.floor(np.nanmin(signlogE_arr))
    vmax = +10 #np.ceil(np.nanmax(signlogE_arr))

    c = plt.pcolor(X, Y, signlogE_arr, cmap='bwr', vmin=vmin, vmax=vmax, edgecolors='none')
    # shade = plt.pcolor(X, Y, RADDAD_arr.T, cmap='binary', vmin=-1, vmax=1, alpha=.05, edgecolors='face')
    shade = plt.contourf(
        x_mesh, y_mesh, RADDAD_arr.T, 
        hatches=[None, '/////'], level=3, alpha=.05)
    plt.colorbar(c, label=r'$\mathrm{sign}(E)\cdot\log E$')
    plt.ylabel(r'$M_\odot$');
    plt.xlabel(r'$\log(\tau_{-1}-\tau)$')
    plt.gca().invert_xaxis();
    # plt.xlim(-2, -1)
    # plt.colorbar()
    plt.savefig('KDiag/%03d.png'%init_mass, bbox_inches='tight')
    plt.close();
    # break

/tmp/ipykernel_4030197/77300805.py:42: RuntimeWarning: divide by zero encountered in log10
  model_info['logt'] = np.log10(model_info['age'].values[-1]-model_info['age'].values)
/tmp/ipykernel_4030197/77300805.py:59: UserWarning: The following kwargs were not used by contour: 'level'
  shade = plt.contourf(
/tmp/ipykernel_4030197/77300805.py:62: MatplotlibDeprecationWarning: Getting the array from a PolyQuadMesh will return the full array in the future (uncompressed). To get this behavior now set the PolyQuadMesh with a 2D array .set_array(data2d).
  plt.colorbar(c, label=r'$\mathrm{sign}(E)\cdot\log E$')
/tmp/ipykernel_4030197/77300805.py:42: RuntimeWarning: divide by zero encountered in log10
  model_info['logt'] = np.log10(model_info['age'].values[-1]-model_info['age'].values)
/tmp/ipykernel_4030197/77300805.py:59: UserWarning: The following kwargs were not used by contour: 'level'
  shade = plt.contourf(
/tmp/ipykernel_4030197/77300805.py:62: MatplotlibDeprecationWarning: Getting t